과제: 파이썬 머신러닝 완벽 가이드 ch8. 4~5

마감: 4월 12일 금요일 18:30

pg. 487~512 필사하여 깃허브 주소를 댓글로 남겨주세요.

# 8.4 텍스트 분류 실습 - 20 뉴스그룹 분류

내용
* 텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 됨
  * 희소 행렬 분류 잘하는 알고리즘: log reg, linear SVM, Naive Bayes 등
  * 이 중 log reg로 분류
* 텍스트를 정규화 한 뒤 피처 벡터화
* 머신러닝 알고리즘으로 분류를 학습/예측/평가
* 카운트 기반, TF-IDF 기반의 벡터화 예측 성능 비교
* 피처 벡터화를 위한 파라미터
* GridSearchCV 기반의 하이퍼 파라미터 튜닝
* 사이킷런의 Pipeline 객체 통해 피처 벡터화 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝을 한꺼번에 수행하는 방법

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset="all", random_state = 156)

## 텍스트 정규화

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd
print("target 클래스의 값과 분포도 \n", pd.Series(news_data.target).value_counts().sort_index())
print("target 클래스의 이름들\n", news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
Name: count, dtype: int64
target 클래스의 이름들
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


* Target 클래스의 값은 0부터 19까지 20개
* 개별 데이터가 텍스트로 어떻게 구성돼 있는지 데이터 한개 만 추출해 확인

In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

헤더와 푸터 정보를 포함하는 것은 의도를 벗어나기에 기사내용으로만 분류할 것.

* remove 파라미터로 header, footer 제거
* subset 파라미터를 이용해 train test 분리해 내려받을 수 있음

In [6]:
from sklearn.datasets import fetch_20newsgroups


train_news = fetch_20newsgroups(subset="train",
                                remove=("headers","footers","quotes"),
                                random_state=156)
test_news = fetch_20newsgroups(subset="test",
                               remove=("headers","footers","quotes"),
                                random_state=156)

X_train = train_news.data
y_train = train_news.target
X_test = test_news.data
y_test = test_news.target

In [7]:
print("학습 데이터 크기 {0}, 텍스트 데이터 크기 {1}".format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 텍스트 데이터 크기 7532


## 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

* CountVectorizer 이용해 학습 데이터의 텍스트를 피처 벡터화
* 테스트 데이터 피처 벡터화 시 유의할 점:
  * 학습 데이터를 이용해 fit이 수행된 CountVectorizer 객체를 이용해서 변환해야 함\
(그래야만 학습 시 설정된 CountVectorizer 피처 개수와 피처개수가 같아짐)
  * fit_transform() 사용하면 안됨
  * transform() 사용

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 텍스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)


print("학습 데이터 텍스트의 CountVectorizer Shape:", X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)

print("CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,pred)))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CountVectorized Logistic Regression의 예측 정확도는 0.606


* Count 기반에서 TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print("TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


* TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공
* 일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출

텍스트 분석에서 모델 성능 향상시키는 중요한 두가지 방법:
1. 최적의 ML 알고리즘을 선택하는 것
2. 최상의 피처 전처리를 수행하는 것
  * 텍스트 정규화
  * Count or TF-IDF 기반 피처 벡터화

In [12]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용.

tfidf_vect = TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print("TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


GridSearchCV 에서 로지스틱 회귀의 최적의 C값 찾기

In [13]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 폴드 세트로 설정.
params = {"C":[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring="accuracy", verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print("Logistic Regression best C parameter :", grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print("TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,pred)))


Fitting 3 folds for each of 5 candidates, totalling 15 fits


KeyboardInterrupt: 

약간 성능 향상

## Pipeline 사용 & GridSearchCV와 결합

* 피처 벡터화, ML 알고리즘 학습/예측을 위한 코드 작성을 한번에 진행 가능케 함
* 데이터의 전처리와 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 모델 코드 생성 가능
* 수행 시간 절약 가능


In [ ]:
from sklearn.pipeline import Pipeline

#TFidVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline생성
pipeline = Pipeline([
    ("tfidf_vect", TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_df=300)),
    ("lr_clf", LogisticRegression(C=10))
])

#별도의 TFidVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요없음
#pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능

pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)

print("Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,pred)))

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("tfidf_vect", TfidfVectorizer(stop_words="english")),
    ("lr_clf", LogisticRegression())
])

params = {"tfidf_vect__ngram_range":[(1,1),(1,2),(1,3)],
         "tfidf_vect__max_df":[100,300,700],
         "lr_clf__C":[1,5,10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline,param_grid=params, cv=3, scoring="accuracy",verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_,grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print("Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,pred)))

실행 실패
> You currently have zero compute units available. Resources offered free of charge are not guaranteed. Purchase more units here.
At your current usage level, this runtime may last up to 113 hours 50 minutes.



Pipeline + GridSearchCV 적용 시 유의할 점: 큰 튜닝 시간 소모

# 8.5 감성 분석

* 감성 분석은 문서 내 주관적인 감성, 의견, 감정, 기분 등을 이해하고 파악하는 방법
* 감성 수치는 긍정 감성 지수와 부정 감성 지수로 구성됨
* 지수를 합산해 긍정/부정 감성 결성
* 지도/비지도 학습 방식으로 나눌 수 있음
  * 지도학습: 학습 데이터와 타깃 레이블을 활용하여 감성 분석을 학습
  * 비지도학습: "Lexicon"이라는 감성 어휘 사전을 활용하여 문서의 긍정적 또는 부정적 감성을 평가 Lexicon은 감성 분석을 위한 다양한 정보를 담고 있어, 감성 어휘와 문맥을 통해 감성을 분석

## Supervised learning 기반 감성 분석 실습 - IMDB 영화평

In [15]:
# Mount your Google Drive.
from google.colab import drive
drive.mount("/content/drive")
kaggle_creds_path = "/content/drive/My Drive/kaggle.json"

! pip install kaggle --quiet
! mkdir ~/.kaggle
! cp "/content/drive/My Drive/kaggle.json" ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [17]:
! kaggle competitions download -c word2vec-nlp-tutorial

 79% 41.0M/51.7M [00:00<00:00, 66.0MB/s]
100% 51.7M/51.7M [00:00<00:00, 67.8MB/s]


In [18]:
! unzip {"word2vec-nlp-tutorial.zip"} -d kaggle_data

Archive:  word2vec-nlp-tutorial.zip
  inflating: kaggle_data/labeledTrainData.tsv.zip  
  inflating: kaggle_data/sampleSubmission.csv  
  inflating: kaggle_data/testData.tsv.zip  
  inflating: kaggle_data/unlabeledTrainData.tsv.zip  


In [20]:
! unzip {"kaggle_data/labeledTrainData.tsv.zip"} -d kaggle_data
! unzip {"kaggle_data/unlabeledTrainData.tsv.zip"} -d kaggle_data

Archive:  kaggle_data/labeledTrainData.tsv.zip
  inflating: kaggle_data/labeledTrainData.tsv  
Archive:  kaggle_data/unlabeledTrainData.tsv.zip
  inflating: kaggle_data/unlabeledTrainData.tsv  


In [21]:
import pandas as pd
review_df = pd.read_csv('kaggle_data/labeledTrainData.tsv',
                        header=0,
                        sep="\t",
                        quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [22]:
print(review_df["review"][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

HTML 형식에서 추출해 \<br/> 태그가 여전히 존재.\
필요없기 때문에 삭제: replace 이용해 공백으로 대체

In [23]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df["review"]=review_df["review"].str.replace("<br />"," ")

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df["review"]=review_df["review"].apply(lambda x : re.sub("[^a-zA-Z]"," ",x))

In [24]:
from sklearn.model_selection import train_test_split

class_df = review_df["sentiment"]
feature_df = review_df.drop(["id","sentiment"], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3,
                                                   random_state=156)

In [25]:
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [27]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [28]:
# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization 수행.


pipeline = Pipeline([
    ("cnt_vect", CountVectorizer(stop_words="english", ngram_range=(1,2))),
    ("lr_clf", LogisticRegression(C=10))
])

In [29]:
# Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc 때문에 수행

pipeline.fit(X_train["review"],y_train)
pred = pipeline.predict(X_test["review"])
pred_probas = pipeline.predict_proba(X_test["review"])[:,1]

print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}".format(accuracy_score(y_test, pred),
                                                roc_auc_score(y_test, pred_probas)))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


예측 정확도는 0.8860, ROC-AUC는 0.9503


`예측 정확도는 0.8860, ROC-AUC는 0.9503`

이번에는 TF-IDF 벡터화를 적용해 다시 예측성능 측정

In [30]:
pipeline = Pipeline([
    ("tfidf_vect", TfidfVectorizer(stop_words="english", ngram_range=(1,2))),
    ("lr_clf", LogisticRegression(C=10))
])

In [31]:
pipeline.fit(X_train["review"],y_train)
pred = pipeline.predict(X_test["review"])
pred_probas = pipeline.predict_proba(X_test["review"])[:,1]

print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}".format(accuracy_score(y_test, pred),
                                                roc_auc_score(y_test, pred_probas)))

예측 정확도는 0.8936, ROC-AUC는 0.9598


`예측 정확도는 0.8936, ROC-AUC는 0.9598`

TF-IDF 기반으로한 게 성능이 조금 더 나아짐

## Unsupervised learning 기반 감성 분석

* Lexicon (어휘집) 기반
* 지도 감성 분석에서는 데이터 세트가 레이블 값을 가지고 있음
* BUT 많은 감성 분석용 데이터는 이렇게 결정된 레이블 값이 없음
* 한글을 지원하는 Lexicon이 없음


* 감정 지수 polarity score: 단어의 위치나 주변 단어, 문맥, Part of Speech (POS) 등을 참고해 결정됨
* 대표적으로 NLTK 패키지
  * Lexicon 모듈 포함
* NLP 패키지의 WordNet: 시맨틱 (문맥상 의미) 분석을 제공하는 어휘 사전

대표적인 감성 사전
* NLTK
* SentiWordNet
* VADER
Pattern

## SentiWordNet 이용한 감성분석


WordNet 기반의 synset을 이용함


In [32]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

'present' 단어에 대한 synset을 추출

In [33]:
from nltk.corpus import wordnet as wn

term ="present"

# "present"라는 단어로 wordnet의 synsets 생성
synsets = wn.synsets(term)
print("synsets() 반환 type :", type(synsets))
print("synsets() 반환 값 개수:", len(synsets))
print("synsets() 반환 값:", synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 개수: 18
synsets() 반환 값: [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


synset 객체가 가지는 속성
* POS
* Definition
* Lemma

In [34]:
for synset in synsets:
    print("#### Synset name:", synset.name(),"####")
    print("POS:", synset.lexname())
    print("Definition:", synset.definition())
    print("Lemmas:", synset.lemma_names())

#### Synset name: present.n.01 ####
POS: noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
#### Synset name: present.n.02 ####
POS: noun.possession
Definition: something presented as a gift
Lemmas: ['present']
#### Synset name: present.n.03 ####
POS: noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
#### Synset name: show.v.01 ####
POS: verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
#### Synset name: present.v.02 ####
POS: verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
#### Synset name: stage.v.01 ####
POS: verb.creation
Definition: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 'represent']
#### Synset name: present.

WordNet은 어휘 간의 관계를 유사도를 나타낼 수 있음: path_similarity()

In [35]:
# synset 객체를 단어별로 생성합니다.

tree = wn.synset("tree.n.01")
lion = wn.synset("lion.n.01")
tiger = wn.synset("tiger.n.02")
cat = wn.synset("cat.n.01")
dog = wn.synset("dog.n.01")
entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split(".")[0] for entity in entities]

# 단어별 synset을 반복하면서 다른 단어의 synset과 유사도를 측정합니다.
for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity),2)
                 for compared_entity in entities]
    similarities.append(similarity)


# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame 형태로 저장
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [36]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets("slow"))
print("senti_sysets() 반환 type:", type(senti_synsets))
print("senti_synsets() 반환 값 개수:", len(senti_synsets))
print("senti_synsets() 반환 값:", senti_synsets)

senti_sysets() 반환 type: <class 'list'>
senti_synsets() 반환 값 개수: 11
senti_synsets() 반환 값: [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [37]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset("father.n.01")
print("father 긍정감성 지수: ", father.pos_score())
print("father 부정감성 지수: ", father.neg_score())
print("father 객관성 지수: ", father.obj_score())
print("--------------------------------------------")
fabulous = swn.senti_synset("fabulous.a.01")
print("fabulous 긍정감성 지수: ", fabulous.pos_score())
print("fabulous 부정감성 지수: ", fabulous.neg_score())
print("fabulous 객관성 지수: ", fabulous.obj_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0
--------------------------------------------
fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125
fabulous 객관성 지수:  0.0


### SentiWordNet을 이용한 영화 감상평 감성 분석

In [43]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환

def penn_to_wn(tag):
    if tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("R"):
        return wn.ADV
    elif tag.startswith("V"):
        return wn.VERB

In [44]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0

    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)

    #분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:
        #NLTK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:

            #WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            #어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            #sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            #모든 단어에 대해 긍정 감성 지수는 +로 부정 감성지수는 -로 합산해 감성 지수 계산
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score()-swn_synset.neg_score())
            tokens_count += 1

    if not tokens_count:
        return 0

    #총 score가 0 이상일 경우 긍정 1, 그렇지 않을 경우 부정 0 반환
    if sentiment>=0:
        return 1
    return 0

In [41]:
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,With all this stuff going down at the moment ...
1,"""2381_9""",1,The Classic War of the Worlds by Timothy ...
2,"""7759_3""",0,The film starts with a manager Nicholas Bell...


In [45]:
review_df["preds"]=review_df["review"].apply(lambda x : swn_polarity(x))
y_target = review_df["sentiment"].values
preds = review_df["preds"].values
review_df.head(3)

,id,sentiment,review,preds
0,"""5814_8""",1,With all this stuff going down at the moment ...,0
1,"""2381_9""",1,The Classic War of the Worlds by Timothy ...,1
2,"""7759_3""",0,The film starts with a manager Nicholas Bell...,0


In [46]:
# SentiWordNet의 감성 분석 예측 성능
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target,preds))
print("정확도:",np.round(accuracy_score(y_target,preds),4))
print("정밀도:",np.round(precision_score(y_target,preds),4))
print("재현율:",np.round(recall_score(y_target,preds),4))

[[7668 4832]
 [3636 8864]]
정확도: 0.6613
정밀도: 0.6472
재현율: 0.7091


## VADER 이용한 감성 분석

* VADER: 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon
* SentimentIntensityAnalyser 클래스를 이용해 쉽게 감성 분석을 제공
* 단독 패키지 모듈 셋업:
```
! pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
```
* LTK 패키지의 서브 모듈
```
import nltk
nltk.download('all')
```
* VADER를 이용하면 매우 쉽게 감성 분석을 수행
* SentimentIntensityAnalyzer 객체를 생성한 뒤에 문서별로 polarity_scores()메서드를 호출해 감성 점수를 구한 뒤, 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 그렇지 않으면 부정으로 판단

In [47]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df["review"][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


In [48]:
def vader_polarity(review, threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)

    #compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
    agg_score = scores["compound"]
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

# apply lambda 식을 이용해 레코드 별로 vader_polarity()를 수행하고 결과를 "vader_preds"에 저장

review_df["vader_preds"]=review_df["review"].apply(lambda x : vader_polarity(x,0.1))
y_target = review_df["sentiment"].values
vader_preds = review_df["vader_preds"].values

print(confusion_matrix(y_target,vader_preds))
print("정확도:",np.round(accuracy_score(y_target,vader_preds),4))
print("정밀도:",np.round(precision_score(y_target,vader_preds),4))
print("재현율:",np.round(recall_score(y_target,vader_preds),4))

[[ 6747  5753]
 [ 1858 10642]]
정확도: 0.6956
정밀도: 0.6491
재현율: 0.8514


* 정확도가 SentiWordNet 보다 향상
* 특히 재현율이 매우 크게 향상
* 이외에도 뛰어난 감성 사전: [pattern](https://github.com/clips/pattern/wiki) 패키지